In [1]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI
import os

In [2]:
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

In [3]:
llm = ChatOpenAI(
    model = "crewai-llama2",
    base_url = "http://localhost:11434/v1")



In [60]:
# Tools

human_tool = load_tools(["human"])

# Image Creation Tool
image_tool = load_tools(["dalle-image-generator"])

In [71]:
# Agents

# Creating a senior researcher agent with memory and verbose mode
manager = Agent(
  name = "Manager",
  #llm = llm,
  role='Senior_Manager',
  goal = (
    'Over see the creation of the game: {topic}. '
    'Plan how to create the game: {topic}. '
    'Organize the tasks that must be completed. '
    'Delegate these tasks to co-workers. '
  ),
  verbose = True,
  memory = True,
  backstory = (
    "Dedicated manager who organizes tasks and delegates them effectively. "
  ),
  allow_delegation = True,
  #tools = human_tool
)

programmer = Agent(
  name = "Programmer",
  #llm = llm,
  role='Senior_Coder',
  goal= (
    'Write perfect python code to create the game: {topic}. '
    'Resolve any bugs or errors in the code. '
    "Do not finalize your answer until the reviewer informs you the game is complete. "
  ),
  verbose=True,
  memory=True,
  backstory=(
    "You create and write perfect python code. "
  ),
  #tools=[search_tool],
  allow_delegation = True
)

# Creating a writer agent with custom tools and delegation capability
tester =Agent(
  name = "Tester",
  #llm = llm,
  role='Tester',
  goal= (
    'Give feedback on the game {topic}, and ensure that the programmer knows of any buggs or errors. '
  ),
  verbose=True,
  memory=True,
  backstory=(
    "Dedicated game tester. "
  ),
  #tools= human_tool,
  allow_delegation = True
)

reviewer = Agent(
    name = "Reviwer",
    role = 'Reviewer',
    goal = (
        "Ensure your coworkers continue working until the human is satisfied. "
        "Ensure that all human feedback has been implemented and the code works as intended. "
    ),
    verbose = True,
    memory = True,
    backstory = (
        "Dedicated to your job you love the human and will make sure your coworkers do their job. "
    ),
    #tools = human_tool,
    allow_delegation = True
)

compiler = Agent(
    name = "Senior_Compiler",
    role = "Code Compiler",
    goal =(
        'Compile code and updates to that code into a code.'
        #'Create a final executable file with the python code for the game {topic} with all the human feedback implemented. '
    ),
    verbose = True,
    memory = True,
    backstory = (
        "You love to read and write code and make sure that all updates are included. "
    )
)
# # Creating artist agent

# artist = Agent(
#   name = "Artist",
#   #llm = llm,
#   role = "Artist",
#   goal = (
#     'Create images for the game: {topic}'
#     'To always refer to yourself as the artist'
#   ),
#   verbose = True,
#   memory = True,
#   backstory = (
#     "Loves to create images used by the programmer"
#     "Always refer to yourself as Artist when speaking"
#     "You always say all the information needed in as few words as possible"
#   ),
#   tools = image_tool,
#   allow_delegation = False
# )

In [72]:
# Tasks

# Manager Task
manager_task = Task(
  description = (
    "Understand the task given. "
    "Plan the exection of the task, and delegate jobs to co-workers. "
  ),
  expected_output = (
    "A python program that runs the game: {topic}. "
    "Do not finalize your answer until the human says the game is complete. "
    #"Ask the human for feed back on the game before finalizing your answer. "
    "Do not finalize your answer until the reviewer recieves confirmation from the human that the game is complete. "
  ),
  agent = manager,
  #output_file = "game.py",
  #human_input = True
)

# Programmer task
programmer_task = Task(
  description=(
    "Write a single file with perfect python code to create the game: {topic}."
    "Take feedback from the tester and reviewer and implement any changes requested. "
    "Do not finalize your answer until the reviewer has recieved confirmation the game is complete. "
  ),
  expected_output=(
    "A single executable file that contains the code for the game {topic}. "
    "The executive file should only contain the code for the game {topic}. "
  ),
  #tools=[search_tool],
  agent= programmer,
  output_file = "game_code.py"
)

# Writing task with language model configuration
tester_task = Task(
  description=(
    "Test the game created by the programmer and report feedback including any bugs and errors. "
    "Give feedback to your co-workers on the results. "
    "Once you can find no more bugs or erros check with the human for feedback before finalizing your answer. "
    "Give the human feedback to your co-workers. "
    "Continue to request feedback from the human then give this feedback to the programmer until the human confirms the game is finished. "
  ),
  expected_output = 'A report to the programmer indicating any bugs or errors in the code.',
  #tools = human_tool,
  agent = tester,
  output_file = 'errors.md' , # Example of output customization
  human_input = False
)

reviewer_task = Task(
    description = (
        #"Make sure all human feedback has been implemented. "
        #"Make sure the code works as intended. "
        "Request confirmation from the human on whether the game is complete. "
        "If the game is not complete tell the manager to implement any neccesarry changes. "
        "If the game is complete tell the programmer they may finalize their answer. "
    ),
    expected_output = 'A report to your co-workers indicating whether all the human feedback has been implemented. ',
    agent = reviewer,
    output_file = "review.md",
    human_input = True,
    #tools = human_tool
)

compiler_task = Task(
    description = (
        "Take the code written by the programmer and add code to implement the human feedback. "
    ),
    expected_output = 'Your Final answer must be the full python code with all buggs and errors fixed and all human feedback implemented. The final answer must only be the python code and nothing else. ',
    agent = compiler,
    output_file = "final_game_code.py"
)

# # Artist task
# artist_task = Task(
#   description = (
#     "Generate images to be used in the game: {topic}. "
#     "Make sure these images are used in the game. "
#   ),
#   expected_output = "Images for the game: {topic}",
#   tools = image_tool,
#   agent = artist,
#   async_execution = False,
#   output_file = 'image.md'
# )

In [74]:
# Crew Creation

# Forming the tech-focused crew with some enhanced configurations
game_crew = Crew(
  agents = [manager, programmer, tester, compiler, reviewer],
  tasks = [manager_task, programmer_task, tester_task, compiler_task, reviewer_task],
  #agents = [programmer],
  #tasks = [programmer_task],
  process=Process.hierarchical,  # Optional: Sequential task execution is default
  manager_llm = ChatOpenAI(model = 'gpt-3.5-turbo'),
  verbose = 2,
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True,
  step_callback = print("STEP ################################################"),
)

2024-06-08 19:48:02,468 - 136140906592064 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


STEP ################################################


In [75]:
if os.path.exists('pong_code.py'):
    os.remove("pong_code.py")
if os.path.exists('pong_game.py'):
    os.remove("pong_game.py")
if os.path.exists('final_pong_code.py'):
    os.remove("final_pong_code.py")
if os.path.exists('errors.md'):
    os.remove("errors.md")
if os.path.exists('review.md'):
    os.remove("review.md")
    
result = game_crew.kickoff(inputs={'topic': 'Mario except with a dinosaur as the character'})

print(result)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Understand the task given. Plan the exection of the task, and delegate jobs to co-workers. 


> Entering new CrewAgentExecutor chain...
I need to understand the task given, plan the execution of the task, and delegate jobs to my co-workers accordingly. I must make sure to provide all necessary context when delegating tasks or asking questions to ensure the best outcome.

Action: Delegate work to co-worker
Action Input: {"task": "Create game framework for Mario with a dinosaur character", "context": "We need to create a Python program that runs the game Mario but with a dinosaur as the character. The game should have the same basic mechanics as the original Mario game, but with the dinosaur as the playable character.", "co-worker": "Senior_Coder"}

> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
```python
import pygame
import sys

# Initialize pygame
pygame.init()

# Set up display
WIDTH